This notebook shows how to use LLM-Geo, you can uncomment a study case (preferred Case 1 and Case 3) to get the spatial analysis results without any human intervention. 

Please check the [webpage of LLM-Geo](https://github.com/gladcolor/LLM-Geo) for more detail!


#  Install package

In [26]:
# ! pip install pyvis
# ! pip install networkx
# ! pip install openai

# Import package

In [27]:
import os
import requests
import networkx as nx
import pandas as pd
import geopandas as gpd
from pyvis.network import Network
from openai import OpenAI
from IPython.display import display, HTML, Code
from IPython.display import clear_output

# Define Solution class
Please run the following cell to define the functions

In [28]:
%load_ext autoreload
%autoreload 2

import LLM_Cat_Constants as constants
import helper

from LLM_Cat_kernel import Solution



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Demonstration Cases

## Input task and data desciption

In [29]:
isReview = True
# isReview = False

In [30]:

# Case 1: population living near hazardous wastes

# task_name ='Resident_at_risk_counting'
# TASK = r"""1) Find out Census tracts that contain hazardous waste facilities, then comppute and print out the population living in those tracts. The study area is North Carolina (NC), US.
# 2) Generate a population choropleth map for all tract polygons in NC, rendering the color by tract population; and then highlight the borders of tracts that have hazardous waste facilities. Please draw all polygons, not only the highlighted ones. The map size is 15*10 inches.
# """

# DATA_LOCATIONS = ["NC hazardous waste facility ESRI shape file: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/HW_Sites_EPSG4326.zip.",
#                   "NC tract boundary shapefile: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/tract_37_EPSG4326.zip. The tract ID column is 'GEOID', data types is integer.",
#                   "NC tract population CSV file: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/NC_tract_population.csv. The population is stored in 'TotalPopulation' column. The tract ID column is 'GEOID', data types is integer."
#                  ]
 


# # Case 3: COVID-19 death rate in US
# task_name ='COVID_death_rate'
# TASK = r'''1) Draw a choropleth map to show the death rate (death/case) of COVID-19 among the countiguous US counties. Use the accumulated COVID-19 data of 2020.12.31 to compute the death rate. Use scheme ='quantiles' when plotting the map.  Set map projection to 'Conus Albers'. Set map size to 15*10 inches.  
# 2) Draw a scatter plot to show the correlation and trend line of the death rate with the senior resident rate, including the r-square and p-value. Set data point transparency to 50%, regression line as red. Set figure size to 15*10 inches.  
# '''


# DATA_LOCATIONS = [
#                   r"COVID-19 data case in 2020 (county-level): https://github.com/nytimes/covid-19-data/raw/master/us-counties-2020.csv. This data is for daily accumulated COVID cases and deaths for each county in the US. There are 5 columns: date (format: 2021-02-01), county, state, fips, cases, deaths. ",   
#                   r"Contiguous US county boundary (ESRI shapefile): https://github.com/gladcolor/spatial_data/raw/master/contiguous_counties.zip. The county FIPS column is 'GEOID'; map projection is EPSG:4269",
#                   r"Census data (ACS2020): https://raw.githubusercontent.com/gladcolor/spatial_data/master/Demography/ACS2020_5year_county.csv. THe needed columns are: 'FIPS', 'Total Population', 'Total Population: 65 to 74 Years', 'Total Population: 75 to 84 Years', 'Total Population: 85 Years and Over'. Drop rows with NaN cells after loading the used columns.",
#                  ]



# # Case 4: Hospital_accessibility
# task_name ='Hospital_accessibility'
# TASK = r'''
# For each zipcode area in South Carolina (SC), calculate the distance from the centroid of the zipcode area to its nearest hospital, and then create a choropleth distance map of zipcode area polygons (unit: km), also show the hospital.
# '''

# DATA_LOCATIONS = [
# r"SC zipcode boundary shapefile: https://github.com/GIBDUSC/test/raw/master/sc_zip_boundary.zip, the map projection is WGS1984.",
# r"SC hospitals:  https://github.com/gladcolor/spatial_data/raw/master/South_Carolina/SC_hospitals_with_emergency_room_cleaned.csv, location columns: longitude in 'POINT_X' column, latitude in 'POINT_Y' column.",          
# ]


## Case 5: Census API
# task_name ='Census_API'
# TASK = r''' Show the spatial distribution of the county level median income in the contigous US. Set figure size to (25,15)
# '''

# DATA_LOCATIONS = [
#       "You can use the Census API.",
# ]

# DATA_LOCATIONS = [
# '''Census API key: ae7be70727932dd6aed257692de3f344365d0678'''              
# ]

# Case 6: Traffic noise
task_name ='Traffic_noise_near_schools'
TASK = r'''1. Download all school polygons from OpenStreetMap for Columbia, South Carolina, USA, including elementary school, middle school, high school, college, university, and so on.
2. Save the downloaded data into ERSI shapefiles in the folder: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Cat\Data.
'''

DATA_LOCATIONS = [

]

 



save_dir = os.path.join(os.getcwd(), task_name)
os.makedirs(save_dir, exist_ok=True)

# create graph
# model=r"gpt-4"
model = r'gpt-4-turbo-preview'
solution = Solution(
                    task=TASK,
                    task_name=task_name,
                    save_dir=save_dir,
                    data_locations=DATA_LOCATIONS,
                    model=model,
                    )
print("Prompt to get solution graph:\n")
print(solution.graph_prompt)

Prompt to get solution graph:

Your role: A professional Geo-information scientist and programmer good at Python. You have worked on Geographic information science more than 20 years, and know every detail and pitfall when processing spatial data and coding. You know well how to set up workflows for spatial analysis tasks. You have significant experence on graph theory, application, and implementation. You are also experienced on generating map using Matplotlib and GeoPandas.
 

Your task: Generate a graph (data structure) only, whose nodes are (1) a series of consecutive steps and (2) data to solve this question:  
 1. Download all school polygons from OpenStreetMap for Columbia, South Carolina, USA, including elementary school, middle school, high school, college, university, and so on.
2. Save the downloaded data into ERSI shapefiles in the folder: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Cat\Data.
 

Your reply needs to meet these requirements: 

## Get graph code from GPT API

In [31]:
response_for_graph = solution.get_LLM_response_for_graph() 
solution.graph_response = response_for_graph
solution.save_solution()

clear_output(wait=True)
display(Code(solution.code_for_graph, language='python'))

import networkx as nx

G = nx.DiGraph()

# Nodes for downloading school polygons from OpenStreetMap for Columbia, South Carolina
G.add_node("OSM_query", node_type="data", path="", description="OpenStreetMap query for schools in Columbia, SC")
G.add_node("download_school_polygons", node_type="operation", description="Download all school polygons from OSM")
G.add_edge("OSM_query", "download_school_polygons")

# Nodes for the downloaded school polygons data
G.add_node("downloaded_school_polygons", node_type="data", path="", description="Downloaded school polygons GeoDataFrame")
G.add_edge("download_school_polygons", "downloaded_school_polygons")

# Nodes for saving the data into ESRI shapefiles
G.add_node("save_schools_shapefile", node_type="operation", description="Save the school polygons data into ESRI shapefiles")
G.add_edge("downloaded_school_polygons", "save_schools_shapefile")

# Final output data
G.add_node("schools_shapefile", node_type="data", path="D:\\OneDrive_PSU\\OneDrive - The Pennsylvania State University\\Research_doc\\LLM-Cat\\Data\\schools.shp", description="ESRI shapefile of the downloaded school polygons")
G.add_edge("save_schools_shapefile", "schools_shapefile")

## Execute code to generate the solution graph

In [32]:
exec(solution.code_for_graph)
solution_graph = solution.load_graph_file()

# Show the graph
G = nx.read_graphml(solution.graph_file)  
nt = helper.show_graph(G)
html_name = os.path.join(os.getcwd(), solution.task_name + '.html')  
# HTML file should in the same directory. See:
# https://stackoverflow.com/questions/65564916/error-displaying-pyvis-html-inside-jupyter-lab-cell
nt.show(name=html_name)
# html_name

D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Cat\Traffic_noise_near_schools.html


## Generate prompts and code for operations (functions)

In [33]:
operations = solution.get_LLM_responses_for_operations(review=isReview)
solution.save_solution()

all_operation_code_str = '\n'.join([operation['operation_code'] for operation in operations])

clear_output(wait=True)
display(Code(all_operation_code_str, language='python'))

import requests
import geopandas as gpd
from shapely.geometry import box

def collect_schools_OSM(OSM_query):
    """
    Description: Collect schools data from OSM using the query for Columbia, SC.
    OSM_query is a dummy parameter to fit the function interface requirement.
    Actual query logic is implemented within the function.
    """
    # Define the bounds for Columbia, South Carolina (min_lon, min_lat, max_lon, max_lat)
    # These coordinates roughly frame the area of interest. Adjust if necessary for more precise coverage.
    bbox = box(-81.2001, 33.9204, -80.886, 34.0861)  # (minx, miny, maxx, maxy)
    
    # Construct Overpass API URL for schools in the defined bounding box
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    (
      node["amenity"="school"]{bbox};
      way["amenity"="school"]{bbox};
      relation["amenity"="school"]{bbox};
    );
    out geom;
    """
    
    response = requests.get(overpass_url, params={'data': overpass_query})
    
    # Check if the request was successful
    if not response.ok:
        raise Exception("Error fetching data from Overpass API")
    
    data = response.json()
    
    # Convert JSON to GeoDataFrame
    schools_data = []
    for element in data['elements']:
        if 'lat' in element and 'lon' in element:
            schools_data.append({
                'type': 'Feature',
                'properties': {'id': element['id'], 'name': element.get('tags', {}).get('name', 'Unknown')},
                'geometry': {
                    'type': 'Point',
                    'coordinates': (element['lon'], element['lat']),
                }
            })
        elif 'geometry' in element:
            schools_data.append({
                'type': 'Feature',
                'properties': {'id': element['id'], 'name': element.get('tags', {}).get('name', 'Unknown')},
                'geometry': {
                    'type': 'Polygon',
                    'coordinates': [[(geom['lon'], geom['lat']) for geom in element['geometry']]],
                }
            })
    
    schools_data_raw = gpd.GeoDataFrame.from_features(schools_data, crs='EPSG:4326')
    
    return schools_data_raw
import geopandas as gpd
import pandas as pd

def clean_and_process_schools_data(schools_data_raw):
    """
    Description: Clean and process raw schools data by removing entries with missing names or geometries
                  and ensuring all geometries are in a consistent EPSG:4326 projection for further processing.
    """
    # Drop rows where 'name' is 'Unknown' or geometry is None
    schools_data_cleaned = schools_data_raw.dropna(subset=['geometry']).copy()
    schools_data_cleaned = schools_data_cleaned[schools_data_cleaned['properties'].apply(lambda x: x['name'] != 'Unknown')]
    
    # Ensure data is in EPSG:4326 projection
    if schools_data_cleaned.crs != "EPSG:4326":
        schools_data_cleaned.to_crs(epsg=4326, inplace=True)
    
    # Additional processing steps could be added here (e.g. normalization of school names, deduplication, etc.)
    
    return schools_data_cleaned
import geopandas as gpd

def convert_to_shp(schools_data_cleaned):
    """
    Description: Convert cleaned schools data to ESRI Shapefile format.
    """
    # Define the output file path
    output_file_path = "D:\\OneDrive_PSU\\OneDrive - The Pennsylvania State University\\Research_doc\\LLM-Cat\\Data\\schools.shp"
    
    # Check if the data is in the correct projection
    if schools_data_cleaned.crs != 'EPSG:4326':
        schools_data_cleaned = schools_data_cleaned.to_crs(epsg=4326)
    
    # Save the GeoDataFrame as a Shapefile
    schools_data_cleaned.to_file(output_file_path)

    return output_file_path

## Generate prompts and code for assembly program

In [34]:
assembly_LLM_response = solution.get_LLM_assembly_response(review=isReview)
solution.assembly_LLM_response = assembly_LLM_response
solution.save_solution()

clear_output(wait=True)
display(Code(solution.code_for_assembly, language='python'))

import requests
import geopandas as gpd
from shapely.geometry import box, Polygon
import pandas as pd

def collect_schools_OSM(OSM_query):
    bbox = box(-81.2001, 33.9204, -80.886, 34.0861)  
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    (
      node["amenity"="school"]{bbox.wkt};
      way["amenity"="school"]{bbox.wkt};
      relation["amenity"="school"]{bbox.wkt};
    );
    out geom;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    if not response.ok:
        raise Exception("Error fetching data from Overpass API")
    
    data = response.json()
    schools_data = []
    for element in data['elements']:
        if 'lat' in element and 'lon' in element:
            schools_data.append({
                'type': 'Feature',
                'properties': {'id': element['id'], 'name': element.get('tags', {}).get('name', 'Unknown')},
                'geometry': {
                    'type': 'Point',
                    'coordinates': (element['lon'], element['lat']),
                }
            })
        elif 'geometry' in element and element['type'] == 'way':
            schools_data.append({
                'type': 'Feature',
                'properties': {'id': element['id'], 'name': element.get('tags', {}).get('name', 'Unknown')},
                'geometry': {
                    'type': 'Polygon',
                    'coordinates': [[(geom['lon'], geom['lat']) for geom in element['geometry']]],
                }
            })
    
    schools_data_raw = gpd.GeoDataFrame.from_features(schools_data, crs='EPSG:4326')
    return schools_data_raw

def clean_and_process_schools_data(schools_data_raw):
    schools_data_cleaned = schools_data_raw.dropna(subset=['geometry']).copy()
    schools_data_cleaned = gpd.GeoDataFrame(
        pd.DataFrame(schools_data_cleaned.drop(columns='properties').join(pd.json_normalize(schools_data_cleaned['properties']))),
        geometry=schools_data_cleaned.geometry, 
        crs=schools_data_cleaned.crs
    )
    schools_data_cleaned = schools_data_cleaned[schools_data_cleaned['name'] != 'Unknown']
    return schools_data_cleaned

def convert_to_shp(schools_data_cleaned):
    output_file_path = "D:\\OneDrive_PSU\\OneDrive - The Pennsylvania State University\\Research_doc\\LLM-Cat\\Data\\schools.shp"
    schools_data_cleaned.to_file(output_file_path)
    return output_file_path

def assembely_solution():
    OSM_query = '' 
    schools_data_raw = collect_schools_OSM(OSM_query)
    schools_data_cleaned = clean_and_process_schools_data(schools_data_raw)
    output_file_path = convert_to_shp(schools_data_cleaned)
    print(f"Data saved to {output_file_path}")

assembely_solution()

## Execute assembly code

In [35]:
all_code = all_operation_code_str + '\n' + solution.code_for_assembly

# display(Code(all_code, language='python'))

all_code = solution.execute_complete_program(code=all_code, try_cnt=10)



-------------- Running code (trial # 1/10) --------------


Error_info_str: 
Traceback (most recent call last):
  File "Complete program", line 164, in <module>
    assembely_solution()
  File "Complete program", line 159, in assembely_solution
    schools_data_raw = collect_schools_OSM(OSM_query)
  File "Complete program", line 115, in collect_schools_OSM
    raise Exception("Error fetching data from Overpass API")
Exception: Error fetching data from Overpass API

Sending error information to LLM for debugging...
```python
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import box, Polygon

def collect_schools_OSM(OSM_query):
    """
    Description: Collect schools data from OSM using the query for Columbia, SC.
    OSM_query is a dummy parameter to fit the function interface requirement.
    Actual query logic is implemented within the function.
    """
    bbox = box(-81.2001, 33.9204, -80.886, 34.0861)  # (min_lon, min_lat, max_lon, max_lat)
   

KeyboardInterrupt: 

In [ ]:
# ! pip install osmnx

In [ ]:
import osmnx as ox
import geopandas as gpd

# Configure osmnx to retrieve points of interest (POIs)
ox.config(use_cache=True, log_console=True)

# Define the location and the query
location = "Columbia, South Carolina, USA"
tags = {'amenity': 'school'}

# Retrieve POIs
schools = ox.geometries_from_place(location, tags)

# Check the retrieved data
print(schools.head())
